<h3>Content</h3>
<p>
1. Building a Graph<br>
2. Different ways of using Session<br>
3. Creating a constant using tf.constant<br>
4. Different way of evaluating a tensor => sess.run(), sess.run().decode(), tensor_variable.eval()<br>
5. Creating a variable<br>
6. Initialising a variable<br>
7. Assigning a value to a variable<br>
8. Very important difference between variable and constant in tensorflow. Also when tf.global_variables_initiailizer() fails : https://stackoverflow.com/questions/44745855/tensorflow-variables-and-constants
</p>

In [1]:
import tensorflow as tf

# Building a graph. A default graph is being built.

# A constant string
msg = tf.constant("Hello, There!!") # Returns a tensor of dtype=string And shape = ()
print(msg) # Tensor("Const:0", shape=(), dtype=string)

# A constant integer
num1 = tf.constant(5) # Returns a tensor of dtype=int32 And shape = ()
print(num1) # Tensor("Const_1:0", shape=(), dtype=int32)

# A constant float
num3 = tf.constant(4.3)
print(num3)

# A list of integers


# Run the graph in a Session
sess = tf.Session()

# Printing a constant String
print(sess.run(msg)) #b'Hello, There!!' => It is a byte array. Happens only with Strings
print(type(sess.run(msg))) # <class 'bytes'>
print(sess.run(msg).decode) # <built-in method decode of bytes object at 0x000002D109F2C300>
print(sess.run(msg).decode()) # Hello, There!!
print(type(sess.run(msg).decode())) # <class 'str'>

# Printing a constant integer
print(sess.run(num1)) # 5


sess.close()

Tensor("Const:0", shape=(), dtype=string)
Tensor("Const_1:0", shape=(), dtype=int32)
Tensor("Const_2:0", shape=(), dtype=float32)
Hello, There!!
<type 'str'>
<built-in method decode of str object at 0x7f229f3eb960>
Hello, There!!
<type 'unicode'>
5


<h3> Evaluating an expression </h3>

In [6]:
# Perform the following : [8*(5+3)]**2 => intermediate_sum = 8, intermediate_product = 8*8 = 64, result = 64*64 = 4096
num1 = tf.constant(5)
num2 = tf.constant(3)
intermediate_sum = tf.add(num1, num2) # Returns a tensor
num3 = tf.constant(8)
intermediate_product = tf.multiply(num3, intermediate_sum)
result = tf.pow(intermediate_product, 2)

In [7]:
with tf.Session() as sess:
    
    # Method 1 : works all the time
    print(sess.run(result)) # While using with tf.Session() as session, we can use result.eval() instead of sess.run(result)
    
    # Method 2 : eval works when we use "with tf.Session() as sess". Also works if we are using Interactive session
    print(result.eval)
    print(result.eval())

4096
<bound method Tensor.eval of <tf.Tensor 'Pow_1:0' shape=() dtype=int32>>
4096


<h2>2 cases when tensor.eval works</h2>

<h3> Case 1 : with tf.Session as sess </h3>
<p> Involves <b>implicit</b> closing of session </p>

In [9]:
with tf.Session() as sess:
    
    # Method 1 : sess.run works all the time
    print(sess.run(result)) # While using with tf.Session() as session, we can use result.eval() instead of sess.run(result)
    
    # Method 2 : eval works when we use "with tf.Session() as sess". Also works if we are using Interactive session
    print(result.eval)
    print(result.eval())
    
    # Default closing of session takes place.

4096
<bound method Tensor.eval of <tf.Tensor 'Pow_1:0' shape=() dtype=int32>>
4096


<h3> Case 2 : tf.InteractiveSession() </h3>
<p> Requires <b>explicit</b> closing of session </p>

In [10]:
sess = tf.InteractiveSession()
print(result.eval())
sess.close() # Explicitly close the session now

4096


<h2>Multiplying 2 Matrices</h2>

In [11]:
mat1 = tf.constant([3,4,3,4], shape=[2,2])
mat2 = tf.constant([4,5,4,5], shape=[2,2])
dot_product = tf.matmul(mat1, mat2)
with tf.Session() as sess: # No explicitly sess.close() required.
    print("---mat1---")
    print(mat1.eval())
    print("---mat2---")
    print(mat2.eval())
    print("--Matrix multiplication---")
    print(dot_product.eval())

---mat1---
[[3 4]
 [3 4]]
---mat2---
[[4 5]
 [4 5]]
--Matrix multiplication---
[[28 35]
 [28 35]]


<h2> Using placeholder to hold variable values </h2>

In [3]:
# Building a Graph
num1 = tf.placeholder(dtype=tf.int32)
num2 = tf.placeholder(dtype=tf.int32)
s = tf.add(num1, num2)

# Taking user input
n1 = int(input("Enter first number: "))
n2 = int(input("Enter second number: "))

# Running the graph in a session
sess = tf.InteractiveSession()# tensor.eval can be used. Explicit closing of session using sess.close() required.
print(s.eval(feed_dict={num1:n1, num2:n2}))
print(sess.run(s,feed_dict={num1:n1, num2:n2}))
sess.close()

Enter first number: 5
Enter second number: 6
11
11


<h2>Initialising Variables using tf.Variable and using tf.global_variables_initializer()</h2>
<p> Note that tf.Variable() constructor ALWAYS creates new variables, even if a variable with same name already exists. It will over-write the previous variable having the same name.
    To reuse a variable, use <b>tf.get_variable()</b>
    </p>

In [29]:
int_value = 53
var1 = tf.Variable(int_value, name='myVariable1') # determines tensor type depending on the type of the value passed.
print("integer variable", var1)
str_value = 'What up!!!'
var2 = tf.Variable(str_value, name='myVariable2')
print("string variable", var2)

with tf.Session() as sess:
   # print(var1.eval())
    '''FailedPreconditionError: Attempting to use uninitialized value myVariable1_4
	 [[Node: _retval_myVariable1_4_0_0 = _Retval[T=DT_INT32, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](myVariable1_4)]]'''
    sess.run(tf.global_variables_initializer()) # Initialize all the variables
    print(var1.eval()) # Not an error.


integer variable <tf.Variable 'myVariable1_8:0' shape=() dtype=int32_ref>
string variable <tf.Variable 'myVariable2_6:0' shape=() dtype=string_ref>
53


<h2>Assigning values to Variables using tf.assign

In [30]:
initial_value_one = 20
v1 = tf.Variable(initial_value_one, name='v1')

initial_value_two = 30
v2 = tf.Variable(initial_value_two, name='v2')

# Taking input from user
value1 = int(input("Enter value 1: "))
value2 = int(input("Enter value 2: "))

# Since we are assigning a value now, it is not necessary to use tf.global_variables_initializer()
t1 = tf.assign(v1, value1)
t2 = tf.assign(v2, value2)

with tf.Session() as sess:
    print(t1.eval())
    print(t2.eval())

Enter value 1: 4
Enter value 2: 3
4
3
